# 🔊 Módulo 4: Síntesis de Voz con Amazon Polly

**Duración**: 15 minutos  
**Objetivo**: Aprender a convertir texto a voz usando IA

## 🎯 ¿Qué aprenderás?
- Convertir texto científico a audio natural
- Crear alertas de voz automáticas
- Usar diferentes voces y idiomas
- Aplicar síntesis de voz a casos científicos
- Personalizar parámetros de síntesis
- Crear contenido educativo accesible

## 🔧 Configuración Inicial

In [ ]:
# Instalar dependencias
import sys
!{sys.executable} -m pip install boto3 ipywidgets --quiet

print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import boto3
import base64
import ipywidgets as widgets
from IPython.display import display, Audio, HTML
import io

print("✅ Librerías importadas")

## 🔑 Configurar Credenciales AWS

In [ ]:
# Configurar Polly
try:
    polly = boto3.client('polly', region_name='us-east-1')
    # Probar conexión
    polly.describe_voices(MaxItems=1)
    print("✅ Cliente Polly configurado")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Configurar credenciales en la siguiente celda")

In [ ]:
# Configuración manual
AWS_ACCESS_KEY_ID = "TU_ACCESS_KEY_AQUI"
AWS_SECRET_ACCESS_KEY = "TU_SECRET_KEY_AQUI"

if AWS_ACCESS_KEY_ID != "TU_ACCESS_KEY_AQUI":
    polly = boto3.client(
        'polly',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-1'
    )
    print("✅ Credenciales configuradas manualmente")
else:
    print("⚠️ Usar credenciales por defecto")

## 📊 Textos Científicos para Síntesis

In [ ]:
# Textos para convertir a voz
POLLY_TEXTS = {
    "Alerta Sísmica": {
        "text": "Atención: Se ha registrado actividad sísmica de magnitud 5.8 en la región costera. Mantenga la calma y siga los protocolos de seguridad establecidos. Aléjese de ventanas y busque refugio seguro.",
        "voice": "Lupe",
        "language": "es-ES"
    },
    
    "Descripción Volcánica": {
        "text": "Los Andes peruanos albergan volcanes activos que requieren monitoreo constante. El Sabancaya y el Ubinas son los más vigilados por su actividad reciente y proximidad a poblaciones.",
        "voice": "Miguel",
        "language": "es-ES"
    },
    
    "Reporte en Inglés": {
        "text": "Seismic monitoring stations have detected increased activity in the southern volcanic region. Enhanced surveillance protocols are now in effect for early warning systems.",
        "voice": "Joanna",
        "language": "en-US"
    },
    
    "Instrucciones de Seguridad": {
        "text": "En caso de sismo: Uno, mantenga la calma. Dos, busque refugio bajo escritorios o marcos de puertas. Tres, aléjese de ventanas. Cuatro, evacúe ordenadamente cuando termine el movimiento.",
        "voice": "Penélope",
        "language": "es-ES"
    },
    
    "Explicación Geológica": {
        "text": "La tectónica de placas es el proceso que explica la formación de montañas, volcanes y terremotos. Las placas de Nazca y Sudamericana interactúan constantemente, generando la actividad sísmica que caracteriza la región andina.",
        "voice": "Lupe",
        "language": "es-ES"
    },
    
    "Alerta Tsunami": {
        "text": "Alerta de tsunami en efecto. Evacúe inmediatamente las zonas costeras y diríjase a terrenos elevados. Manténgase alejado de la playa y ríos. Siga las rutas de evacuación señalizadas.",
        "voice": "Miguel",
        "language": "es-ES"
    },
    
    "Contenido Educativo": {
        "text": "Los minerales son sustancias naturales con composición química definida y estructura cristalina ordenada. El cuarzo, la calcita y el feldespato son algunos de los minerales más comunes en la corteza terrestre.",
        "voice": "Penélope",
        "language": "es-ES"
    }
}

print("🔊 Textos para síntesis cargados:")
for i, (title, data) in enumerate(POLLY_TEXTS.items(), 1):
    word_count = len(data['text'].split())
    print(f"  {i}. {title} ({word_count} palabras, voz: {data['voice']})")

## 🔍 Función de Síntesis de Voz

In [ ]:
def synthesize_speech(text, voice_id, language_code, text_name, use_ssml=False, speech_rate=100, volume=100):
    """Sintetiza voz usando Amazon Polly con soporte avanzado"""
    try:
        print(f"🔄 Sintetizando: {text_name}")
        print(f"  🎤 Voz: {voice_id}")
        print(f"  🌍 Idioma: {language_code}")
        
        # Verificar longitud del texto
        if len(text) > 3000:
            print(f"⚠️ Texto largo ({len(text)} caracteres), dividiendo en chunks...")
            return synthesize_long_text(text, voice_id, language_code, text_name)
        
        # Preparar texto con SSML si se solicita
        if use_ssml:
            ssml_text = f"""<speak>
                <prosody rate="{speech_rate}%" volume="{volume}%">
                    {text}
                </prosody>
            </speak>"""
            text_type = 'ssml'
            final_text = ssml_text
            print(f"  🎚️ SSML: Velocidad {speech_rate}%, Volumen {volume}%")
        else:
            text_type = 'text'
            final_text = text
        
        # Llamar a Polly
        response = polly.synthesize_speech(
            Text=final_text,
            TextType=text_type,
            OutputFormat='mp3',
            VoiceId=voice_id,
            LanguageCode=language_code
        )
        
        # Obtener audio
        audio_stream = response['AudioStream']
        audio_data = audio_stream.read()
        
        print(f"✅ Audio generado: {len(audio_data)} bytes")
        
        return {
            'success': True,
            'audio_data': audio_data,
            'text_name': text_name,
            'voice_id': voice_id,
            'language_code': language_code,
            'text': text,
            'use_ssml': use_ssml
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        # Simulación si Polly no está disponible
        print("🔄 Usando simulación de síntesis de voz...")
        
        # Crear audio simulado (silencio)
        import wave
        import struct
        
        # Parámetros de audio
        sample_rate = 22050
        duration = min(len(text.split()) * 0.5, 10)  # ~0.5 segundos por palabra, máximo 10 seg
        
        # Generar silencio
        num_samples = int(sample_rate * duration)
        audio_data = b'\x00' * (num_samples * 2)  # 16-bit audio
        
        print(f"🔇 Audio simulado generado: {len(audio_data)} bytes ({duration:.1f}s)")
        
        return {
            'success': True,
            'audio_data': audio_data,
            'text_name': text_name,
            'voice_id': voice_id,
            'language_code': language_code,
            'text': text,
            'simulated': True
        }

def synthesize_long_text(text, voice_id, language_code, text_name):
    """Maneja textos largos dividiéndolos en chunks"""
    try:
        # Dividir texto en oraciones
        sentences = text.replace('.', '.\n').replace('!', '!\n').replace('?', '?\n').split('\n')
        sentences = [s.strip() for s in sentences if s.strip()]
        
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk + sentence) < 2500:  # Límite seguro
                current_chunk += sentence + " "
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
        
        if current_chunk:
            chunks.append(current_chunk.strip())
        
        print(f"📝 Texto dividido en {len(chunks)} chunks")
        
        # Sintetizar cada chunk
        all_audio_data = b''
        for i, chunk in enumerate(chunks, 1):
            print(f"  🔄 Procesando chunk {i}/{len(chunks)}...")
            
            response = polly.synthesize_speech(
                Text=chunk,
                OutputFormat='mp3',
                VoiceId=voice_id,
                LanguageCode=language_code
            )
            
            chunk_audio = response['AudioStream'].read()
            all_audio_data += chunk_audio
        
        print(f"✅ Audio completo generado: {len(all_audio_data)} bytes")
        
        return {
            'success': True,
            'audio_data': all_audio_data,
            'text_name': text_name,
            'voice_id': voice_id,
            'language_code': language_code,
            'text': text,
            'chunks': len(chunks)
        }
        
    except Exception as e:
        print(f"❌ Error en texto largo: {e}")
        return {'success': False, 'error': str(e)}

def get_available_voices():
    """Obtiene las voces disponibles de Polly"""
    try:
        response = polly.describe_voices()
        voices = response['Voices']
        
        # Organizar por idioma
        voices_by_language = {}
        for voice in voices:
            lang = voice['LanguageName']
            if lang not in voices_by_language:
                voices_by_language[lang] = []
            voices_by_language[lang].append({
                'id': voice['Id'],
                'name': voice['Name'],
                'gender': voice['Gender'],
                'language_code': voice['LanguageCode']
            })
        
        print(f"🎤 Voces disponibles en Polly: {len(voices)} total")
        for lang, lang_voices in voices_by_language.items():
            print(f"  • {lang}: {len(lang_voices)} voces")
        
        return voices_by_language
        
    except Exception as e:
        print(f"❌ Error obteniendo voces: {e}")
        # Voces por defecto
        return {
            'Spanish': [
                {'id': 'Lupe', 'name': 'Lupe', 'gender': 'Female', 'language_code': 'es-ES'},
                {'id': 'Miguel', 'name': 'Miguel', 'gender': 'Male', 'language_code': 'es-ES'},
                {'id': 'Penélope', 'name': 'Penélope', 'gender': 'Female', 'language_code': 'es-ES'}
            ],
            'English': [
                {'id': 'Joanna', 'name': 'Joanna', 'gender': 'Female', 'language_code': 'en-US'},
                {'id': 'Matthew', 'name': 'Matthew', 'gender': 'Male', 'language_code': 'en-US'},
                {'id': 'Amy', 'name': 'Amy', 'gender': 'Female', 'language_code': 'en-GB'}
            ]
        }

def synthesize_with_pronunciation(text, voice_id, language_code, text_name, custom_pronunciations=None):
    """Síntesis con pronunciaciones personalizadas usando SSML"""
    try:
        print(f"🔄 Síntesis con pronunciación personalizada: {text_name}")
        
        # Aplicar pronunciaciones personalizadas
        if custom_pronunciations:
            ssml_text = "<speak>"
            current_text = text
            
            for word, pronunciation in custom_pronunciations.items():
                # Reemplazar con etiqueta phoneme
                current_text = current_text.replace(
                    word, 
                    f'<phoneme alphabet="ipa" ph="{pronunciation}">{word}</phoneme>'
                )
            
            ssml_text += current_text + "</speak>"
            
            print(f"  🗣️ Pronunciaciones aplicadas: {list(custom_pronunciations.keys())}")
        else:
            ssml_text = f"<speak>{text}</speak>"
        
        # Llamar a Polly con SSML
        response = polly.synthesize_speech(
            Text=ssml_text,
            TextType='ssml',
            OutputFormat='mp3',
            VoiceId=voice_id,
            LanguageCode=language_code
        )
        
        audio_data = response['AudioStream'].read()
        
        print(f"✅ Audio con pronunciación generado: {len(audio_data)} bytes")
        
        return {
            'success': True,
            'audio_data': audio_data,
            'text_name': text_name,
            'voice_id': voice_id,
            'language_code': language_code,
            'text': text,
            'pronunciations': custom_pronunciations
        }
        
    except Exception as e:
        print(f"❌ Error en pronunciación: {e}")
        # Fallback a síntesis normal
        return synthesize_speech(text, voice_id, language_code, text_name)

def compare_voices(text, voices_list, language_code, comparison_name="Comparación de Voces"):
    """Compara el mismo texto con diferentes voces"""
    print(f"🎭 COMPARACIÓN DE VOCES: {comparison_name.upper()}")
    print(f"📝 Texto: '{text}'")
    print("=" * 70)
    
    results = []
    
    for i, voice_id in enumerate(voices_list, 1):
        print(f"\n🎤 Voz {i}/{len(voices_list)}: {voice_id}")
        print("-" * 40)
        
        result = synthesize_speech(
            text=text,
            voice_id=voice_id,
            language_code=language_code,
            text_name=f"{comparison_name} - {voice_id}"
        )
        
        if result.get('success'):
            results.append(result)
            
            # Reproductor compacto para comparación
            audio_widget = Audio(data=result['audio_data'], autoplay=False)
            print(f"🔊 Reproductor para {voice_id}:")
            display(audio_widget)
            
            # Estadísticas básicas
            audio_size = len(result['audio_data']) / 1024
            print(f"📊 Tamaño: {audio_size:.1f} KB")
    
    # Resumen de comparación
    if results:
        print(f"\n📈 RESUMEN DE COMPARACIÓN:")
        print("=" * 40)
        
        sizes = [len(r['audio_data'])/1024 for r in results]
        voices = [r['voice_id'] for r in results]
        
        print(f"• Voces comparadas: {len(results)}")
        print(f"• Tamaño promedio: {np.mean(sizes):.1f} KB")
        print(f"• Voz con audio más largo: {voices[np.argmax(sizes)]} ({max(sizes):.1f} KB)")
        print(f"• Voz con audio más corto: {voices[np.argmin(sizes)]} ({min(sizes):.1f} KB)")
        
        # Gráfico de comparación
        try:
            import matplotlib.pyplot as plt
            
            plt.figure(figsize=(10, 6))
            bars = plt.bar(voices, sizes, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(voices)], alpha=0.8)
            plt.title(f'Comparación de Tamaños de Audio - {comparison_name}', fontweight='bold')
            plt.xlabel('Voces')
            plt.ylabel('Tamaño (KB)')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            
            # Agregar valores en las barras
            for bar, size in zip(bars, sizes):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                        f'{size:.1f}', ha='center', va='bottom', fontweight='bold')
            
            plt.tight_layout()
            plt.show()
            
        except ImportError:
            print("📊 Gráfico de comparación no disponible (requiere matplotlib)")
    
    return results

def compare_speech_parameters(text, voice_id, language_code, parameter_sets):
    """Compara diferentes parámetros de síntesis (velocidad, volumen)"""
    print(f"🎚️ COMPARACIÓN DE PARÁMETROS DE SÍNTESIS")
    print(f"🎤 Voz: {voice_id}")
    print(f"📝 Texto: '{text}'")
    print("=" * 70)
    
    results = []
    
    for i, params in enumerate(parameter_sets, 1):
        rate = params.get('rate', 100)
        volume = params.get('volume', 100)
        name = params.get('name', f'Configuración {i}')
        
        print(f"\n🎛️ {name}: Velocidad {rate}%, Volumen {volume}%")
        print("-" * 50)
        
        result = synthesize_speech(
            text=text,
            voice_id=voice_id,
            language_code=language_code,
            text_name=f"Parámetros - {name}",
            use_ssml=True,
            speech_rate=rate,
            volume=volume
        )
        
        if result.get('success'):
            results.append({**result, 'rate': rate, 'volume': volume, 'config_name': name})
            
            # Reproductor para cada configuración
            audio_widget = Audio(data=result['audio_data'], autoplay=False)
            print(f"🔊 Reproductor - {name}:")
            display(audio_widget)
    
    return results

print("✅ Funciones de síntesis y comparación avanzadas creadas")

## 📊 Función de Reproducción

In [ ]:
def play_synthesized_audio(synthesis_result, show_waveform=True, enable_download=True):
    """Reproduce el audio sintetizado con controles avanzados"""
    if not synthesis_result.get('success'):
        print(f"❌ Error: {synthesis_result.get('error')}")
        return
    
    text_name = synthesis_result['text_name']
    voice_id = synthesis_result['voice_id']
    audio_data = synthesis_result['audio_data']
    text = synthesis_result['text']
    is_simulated = synthesis_result.get('simulated', False)
    
    print(f"🔊 REPRODUCIENDO: {text_name.upper()}")
    print(f"🎤 Voz: {voice_id}")
    if is_simulated:
        print("⚠️ Modo: Simulación (Polly no disponible)")
    print("=" * 60)
    
    # Mostrar texto original
    print(f"📝 Texto original:")
    print(f"'{text}'")
    print()
    
    # Crear reproductor de audio mejorado
    audio_widget = Audio(data=audio_data, autoplay=False)
    
    print("🎵 Reproductor de audio:")
    display(audio_widget)
    
    # Visualización de forma de onda (si está habilitada)
    if show_waveform and not is_simulated:
        try:
            create_waveform_visualization(audio_data, text_name)
        except Exception as e:
            print(f"⚠️ No se pudo crear visualización de onda: {e}")
    
    # Botón de descarga (si está habilitado)
    if enable_download:
        create_download_button(audio_data, text_name, voice_id)
    
    # Estadísticas detalladas
    word_count = len(text.split())
    char_count = len(text)
    audio_size_kb = len(audio_data) / 1024
    estimated_duration = word_count * 0.5  # ~0.5 segundos por palabra
    
    print(f"\n📊 ESTADÍSTICAS DETALLADAS:")
    print(f"  • Palabras: {word_count}")
    print(f"  • Caracteres: {char_count}")
    print(f"  • Tamaño audio: {audio_size_kb:.1f} KB")
    print(f"  • Duración estimada: {estimated_duration:.1f} segundos")
    print(f"  • Formato: MP3")
    print(f"  • Voz utilizada: {voice_id}")
    
    if synthesis_result.get('chunks'):
        print(f"  • Chunks procesados: {synthesis_result['chunks']}")
    
    if synthesis_result.get('use_ssml'):
        print(f"  • SSML utilizado: Sí")

def create_waveform_visualization(audio_data, title):
    """Crea una visualización de forma de onda del audio"""
    try:
        import numpy as np
        import matplotlib.pyplot as plt
        
        # Simular forma de onda (en un caso real se usaría librosa o similar)
        # Para MP3 necesitaríamos decodificar, aquí simulamos
        duration = len(audio_data) / 16000  # Estimación aproximada
        time = np.linspace(0, duration, 1000)
        
        # Generar forma de onda simulada
        np.random.seed(42)  # Para consistencia
        amplitude = np.random.normal(0, 0.3, 1000)
        amplitude *= np.exp(-time/duration * 2)  # Decay natural
        
        # Crear gráfico
        plt.figure(figsize=(12, 4))
        plt.plot(time, amplitude, color='steelblue', linewidth=0.8)
        plt.fill_between(time, amplitude, alpha=0.3, color='lightblue')
        plt.title(f'Forma de Onda - {title}', fontweight='bold', fontsize=14)
        plt.xlabel('Tiempo (segundos)')
        plt.ylabel('Amplitud')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Agregar información
        plt.text(0.02, 0.95, f'Duración: {duration:.1f}s', transform=plt.gca().transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.show()
        
    except ImportError:
        print("📊 Visualización de onda no disponible (requiere numpy y matplotlib)")
    except Exception as e:
        print(f"⚠️ Error creando visualización: {e}")

def create_download_button(audio_data, text_name, voice_id):
    """Crea un enlace de descarga para el audio"""
    try:
        import base64
        from IPython.display import HTML
        
        # Codificar audio en base64
        audio_b64 = base64.b64encode(audio_data).decode()
        
        # Crear nombre de archivo
        filename = f"{text_name.replace(' ', '_')}_{voice_id}.mp3"
        
        # Crear HTML para descarga
        download_html = f"""
        <div style="margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9;">
            <h4 style="margin: 0 0 10px 0; color: #333;">💾 Descargar Audio</h4>
            <a href="data:audio/mp3;base64,{audio_b64}" download="{filename}" 
               style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                      text-decoration: none; border-radius: 4px; font-weight: bold;">
                📥 Descargar {filename}
            </a>
            <p style="margin: 10px 0 0 0; font-size: 12px; color: #666;">
                Tamaño: {len(audio_data)/1024:.1f} KB | Formato: MP3
            </p>
        </div>
        """
        
        display(HTML(download_html))
        
    except Exception as e:
        print(f"⚠️ Descarga no disponible: {e}")

print("✅ Función de reproducción creada")

## 🚀 Ejercicio 1: Alerta Sísmica

In [ ]:
# Sintetizar alerta sísmica
text_name = "Alerta Sísmica"
text_data = POLLY_TEXTS[text_name]

print(f"🚨 Generando audio: {text_name}")
print("-" * 50)

# Sintetizar
alert_audio = synthesize_speech(
    text=text_data['text'],
    voice_id=text_data['voice'],
    language_code=text_data['language'],
    text_name=text_name
)

# Reproducir
if alert_audio.get('success'):
    play_synthesized_audio(alert_audio)

## 🌋 Ejercicio 2: Descripción Volcánica

In [ ]:
# Sintetizar descripción volcánica
text_name = "Descripción Volcánica"
text_data = POLLY_TEXTS[text_name]

print(f"🌋 Generando audio: {text_name}")
print("-" * 50)

volcanic_audio = synthesize_speech(
    text=text_data['text'],
    voice_id=text_data['voice'],
    language_code=text_data['language'],
    text_name=text_name
)

if volcanic_audio.get('success'):
    play_synthesized_audio(volcanic_audio)

## 🎛️ Widget Interactivo

In [ ]:
# Widget para síntesis interactiva
def interactive_synthesis(text_selection):
    text_data = POLLY_TEXTS[text_selection]
    
    print(f"🔊 Sintetizando: {text_selection}")
    print("=" * 50)
    
    result = synthesize_speech(
        text=text_data['text'],
        voice_id=text_data['voice'],
        language_code=text_data['language'],
        text_name=text_selection
    )
    
    if result.get('success'):
        play_synthesized_audio(result)
    
    return result

# Crear widget
text_selector = widgets.Dropdown(
    options=list(POLLY_TEXTS.keys()),
    description='Texto:'
)

interactive_widget = widgets.interactive(interactive_synthesis, text_selection=text_selector)
display(interactive_widget)

## 🎓 Casos de Uso Reales

### Amazon Polly en Ciencias:
- 🚨 **Alertas automáticas**: Convertir alertas sísmicas a audio
- 📚 **Contenido educativo**: Crear audiolibros científicos
- 🎤 **Presentaciones**: Narrar automáticamente reportes
- 🌍 **Multiidioma**: Comunicar en diferentes idiomas
- ♿ **Accesibilidad**: Hacer contenido accesible para personas con discapacidad visual

## ✅ Validación del Módulo

In [ ]:
def validate_module():
    checks = {
        "Cliente configurado": False,
        "Audio sintetizado": False,
        "Widget usado": False
    }
    
    try:
        polly.describe_voices(MaxItems=1)
        checks["Cliente configurado"] = True
    except:
        pass
    
    if 'alert_audio' in globals() and alert_audio.get('success'):
        checks["Audio sintetizado"] = True
        checks["Widget usado"] = True
    
    print("📋 VALIDACIÓN MÓDULO 4 - POLLY")
    print("=" * 40)
    
    for check, status in checks.items():
        icon = "✅" if status else "❌"
        print(f"{icon} {check}")
    
    completed = sum(checks.values())
    total = len(checks)
    print(f"\n📊 Progreso: {completed}/{total} ({completed/total*100:.0f}%)")
    
    if completed >= 2:
        print("\n🎉 ¡MÓDULO COMPLETADO!")
        print("➡️ Continúa con Módulo 5: Bedrock")
    else:
        print("\n⚠️ Completa los ejercicios faltantes")
    
    return completed >= 2

module_completed = validate_module()

---

## 🚀 Próximo Módulo

**🤖 Módulo 5: IA Generativa con Amazon Bedrock**

---

*Módulo 4 de 6 completado* ✅